In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

We want to eliminate "accidental" referrals, so filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50.

In [ ]:
##db = sqlite3.connect('data/doc_grahp_hop.sqlite')

##chunks = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
##hop = []
##for chunk in chunks: 
##    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
##    hop.append(chunk)
##    chunk.to_sql('hop', db, if_exists = 'append', index = False) 

In [ ]:
##hopp = pd.concat(hop, ignore_index = True)
##hopp


The NPPES dataset contains a large number of fields, only a few of which are relevant to this project:

In [2]:
cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)',
        'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name',
        'Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 
        'Provider First Line Business Practice Location Address', 
        'Provider Second Line Business Practice Location Address',
        'Provider Business Practice Location Address City Name',
        'Provider Business Practice Location Address State Name',
        'Provider Business Practice Location Address Postal Code', 
        'Healthcare Provider Taxonomy Code_1', 
        'Healthcare Provider Primary Taxonomy Switch_1', 
        'Healthcare Provider Taxonomy Code_2', 
        'Healthcare Provider Primary Taxonomy Switch_2', 
        'Healthcare Provider Taxonomy Code_3', 
        'Healthcare Provider Primary Taxonomy Switch_3', 
        'Healthcare Provider Taxonomy Code_4', 
        'Healthcare Provider Primary Taxonomy Switch_4', 
        'Healthcare Provider Taxonomy Code_5', 
        'Healthcare Provider Primary Taxonomy Switch_5', 
        'Healthcare Provider Taxonomy Code_6', 
        'Healthcare Provider Primary Taxonomy Switch_6', 
        'Healthcare Provider Taxonomy Code_7', 
        'Healthcare Provider Primary Taxonomy Switch_7', 
        'Healthcare Provider Taxonomy Code_8', 
        'Healthcare Provider Primary Taxonomy Switch_8', 
        'Healthcare Provider Taxonomy Code_9', 
        'Healthcare Provider Primary Taxonomy Switch_9', 
        'Healthcare Provider Taxonomy Code_10', 
        'Healthcare Provider Primary Taxonomy Switch_10', 
        'Healthcare Provider Taxonomy Code_11', 
        'Healthcare Provider Primary Taxonomy Switch_11',
        'Healthcare Provider Taxonomy Code_12', 
        'Healthcare Provider Primary Taxonomy Switch_12', 
        'Healthcare Provider Taxonomy Code_13', 
        'Healthcare Provider Primary Taxonomy Switch_13', 
        'Healthcare Provider Taxonomy Code_14', 
        'Healthcare Provider Primary Taxonomy Switch_14', 
        'Healthcare Provider Taxonomy Code_15', 
        'Healthcare Provider Primary Taxonomy Switch_15']

nppes_chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2023/npidata_pfile_20050523-20230212.csv', usecols=cols, iterator=True, chunksize=1000)

In [ ]:
state_list = ['TN', 'AR', 'NC', 'MS', 'GA']
nppes = pd.concat([chunk[chunk['Provider Business Practice Location Address State Name'].isin(state_list)] for chunk in nppes_chunks])

In [ ]:
var_cols = ['Healthcare Provider Taxonomy Code', 'Healthcare Provider Primary Taxonomy Switch']

In [ ]:
state_list = ['TN', 'AR', 'NC', 'MS', 'GA']
nppes = pd.DataFrame()
id_cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code']

for chunk in nppes_chunks:
    filtered_chunk = chunk[chunk['Provider Business Practice Location Address State Name'].isin(state_list)]
    filtered_chunk = pd.wide_to_long(filtered_chunk, var_cols, i = id_cols , j = "", sep = "_")
    
    nppes = pd.concat([nppes, filtered_chunk])

In [ ]:
nppes.reset_index()

In [11]:
nppes.head()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,,Healthcare Provider Taxonomy Code,Healthcare Provider Primary Taxonomy Switch
0,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009,1,183500000X,Y
1,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009,2,NaN,NaN
2,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009,3,NaN,NaN
3,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009,4,NaN,NaN
4,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009,5,NaN,NaN


Converting the data into a sqlite database

In [36]:
db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')

cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text',  'Provider Name Suffix Text','Provider Credential Text', 
        'Provider First Line Business Practice Location Address', 
        'Provider Second Line Business Practice Location Address',
        'Provider Business Practice Location Address City Name',
        'Provider Business Practice Location Address State Name',
        'Provider Business Practice Location Address Postal Code', 
        'Healthcare Provider Taxonomy Code_1', 
        'Healthcare Provider Primary Taxonomy Switch_1', 
        'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10', 'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15']

nppes_chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2023/npidata_pfile_20050523-20230212.csv', usecols=cols, iterator=True, chunksize=100000, nrows=500000, low_memory=False)
var_cols = ['Healthcare Provider Taxonomy Code', 'Healthcare Provider Primary Taxonomy Switch']

nppes_full = []

state_list = ['TN', 'AR', 'NC', 'MS', 'GA']
nppes = pd.DataFrame()
id_cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code']

for chunk in nppes_chunks:
    filtered_chunk = chunk[chunk['Provider Business Practice Location Address State Name'].isin(state_list)]
    filtered_chunk = pd.wide_to_long(filtered_chunk, var_cols, i = id_cols , j = "", sep = "_")
    
    nppes = pd.concat([nppes, filtered_chunk])

nppes = nppes.reset_index()
nppes.columns = nppes.columns.str.replace(" ", "")
nppes = nppes.drop(columns = "")


nppes.to_sql('nppes', con = db, if_exists = 'append', index = False) 


db.close()

In [28]:
db.close()

Verifying the data.

In [37]:
db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')
query = "SELECT * FROM nppes"

In [38]:
with sqlite3.connect('data/NPPES_Data_Dissemination.sqlite') as db: 
    nppes_sqlite = pd.read_sql(query, db)

In [39]:
nppes_sqlite

,NPI,EntityTypeCode,ProviderOrganizationName(LegalBusinessName),ProviderLastName(LegalName),ProviderFirstName,ProviderMiddleName,ProviderNamePrefixText,ProviderNameSuffixText,ProviderCredentialText,ProviderFirstLineBusinessPracticeLocationAddress,ProviderSecondLineBusinessPracticeLocationAddress,ProviderBusinessPracticeLocationAddressCityName,ProviderBusinessPracticeLocationAddressStateName,ProviderBusinessPracticeLocationAddressPostalCode,HealthcareProviderTaxonomyCode,HealthcareProviderPrimaryTaxonomySwitch
0,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,183500000X,Y
1,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,None,None
2,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,None,None
3,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,None,None
4,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614035,1134346331,1.0,None,COOKE,JOHN,T,None,None,LPC,139 HENRY PKWY,None,MCDONOUGH,GA,302536636,None,None
614036,1134346331,1.0,None,COOKE,JOHN,T,None,None,LPC,139 HENRY PKWY,None,MCDONOUGH,GA,302536636,None,None
614037,1134346331,1.0,None,COOKE,JOHN,T,None,None,LPC,139 HENRY PKWY,None,MCDONOUGH,GA,302536636,None,None
614038,1134346331,1.0,None,COOKE,JOHN,T,None,None,LPC,139 HENRY PKWY,None,MCDONOUGH,GA,302536636,None,None


Using the primary taxonomy code, match each provider to a classification (from the Classification column

In [ ]:
db = sqlite3.connect('data/nucc_taxonomy.sqlite')

chunks = pd.read_csv('data/nucc_taxonomy_230.csv', chunksize = 10000)


Match each provider to a CBSA using the Business Zip code

In [ ]:
db = sqlite3.connect('data/CBSA.sqlite')
CBSA = pd.read_excel('data/ZIP_CBSA_122021.xlsx')
CBSA